## Task 1 — DP + MDP for Tic-Tac-Toe (Value Iteration)

MDP formulation (X’s perspective):

     1. States: every legal 3×3 board (string length 9 over {X,O,space}), with X to move on non-terminal states.
     
     2. Actions: place X in any empty cell (0–8).
     
     3. Transitions: after X moves, O moves uniformly at random among its legal moves (if X just ended the game, transition is terminal).
     
     4. Rewards: +1 if X wins, −1 if O wins, 0 for draw/ongoing.
     
     5. Discount: γ = 1 (episodic).

Goal: compute V*(s) and the greedy policy π*(s) via Value Iteration.

In [1]:
# task1_ttt_dp.py
from collections import defaultdict
from functools import lru_cache
import random

WIN = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]

def legal(board): return [i for i,c in enumerate(board) if c==' ']
def place(b,i,ch): return b[:i]+ch+b[i+1:]
def cur_player(b): return 'X' if b.count('X')==b.count('O') else 'O'
def is_draw(b): return ' ' not in b and winner(b) is None

@lru_cache(None)
def winner(b):
    for a,b1,c in WIN:
        s = b[a]+b[b1]+b[c]
        if s=='XXX': return 'X'
        if s=='OOO': return 'O'
    return None

def terminal(b): return winner(b) or ' ' not in b

def gen_states():
    start = ' '*9; seen=set()
    def dfs(b):
        if b in seen: return
        seen.add(b)
        if terminal(b): return
        p = cur_player(b)
        for a in legal(b): dfs(place(b,a,p))
    dfs(start)
    return seen

def transitions_after_X(b,a):
    s1 = place(b,a,'X')
    if terminal(s1):
        w = winner(s1); r = 1 if w=='X' else (-1 if w=='O' else 0)
        return [(1.0, s1, r)]
    moves = legal(s1); p = 1.0/len(moves)
    outs=[]
    for om in moves:
        s2 = place(s1,om,'O')
        if terminal(s2):
            w=winner(s2); r = 1 if w=='X' else (-1 if w=='O' else 0)
            outs.append((p,s2,r))
        else:
            outs.append((p,s2,0))
    return outs

def value_iteration(gamma=1.0, tol=1e-10):
    all_s = gen_states()
    xs = [s for s in all_s if (not terminal(s)) and cur_player(s)=='X']
    V = defaultdict(float); policy={}
    while True:
        delta=0.0
        for s in xs:
            qbest=-1e9; abest=None
            for a in legal(s):
                q=0.0
                for p,sn,r in transitions_after_X(s,a):
                    q += p*(r + gamma*V[sn])
                if q>qbest: qbest,abest=q,a
            delta=max(delta,abs(qbest-V[s]))
            V[s]=qbest; policy[s]=abest
        if delta<tol: break
    return V,policy

def pretty(b):
    return '\n'.join(' | '.join(ch if ch!=' ' else '.' for ch in b[i:i+3]) for i in (0,3,6))

def play_policy_vs_random(policy, seed=0):
    rng=random.Random(seed); b=' '*9
    while not terminal(b):
        if cur_player(b)=='X':
            a = policy.get(b, random.choice(legal(b)))
            b = place(b,a,'X')
        else:
            b = place(b, rng.choice(legal(b)),'O')
    return b, winner(b) or ('draw' if is_draw(b) else None)

if __name__=='__main__':
    V,pi = value_iteration()
    b,who = play_policy_vs_random(pi)
    print("Best first move from empty:", pi.get(' '*9))
    print("Final board:\n"+pretty(b))
    print("Outcome:", who)


Best first move from empty: 0
Final board:
X | X | X
. | . | O
. | O | .
Outcome: X


## Task 2 — RL (Tabular Q-Learning) + Performance vs DP

1. Algorithm: Tabular Q-Learning with ε-greedy (ε decays), α learning rate, γ=1.

2. Training: X learns by playing vs a fixed random O (or self-play if you prefer).

3. Evaluation: Plot or table win/draw/loss rates over episodes and compare to DP policy from Task 1.

4. Reflection: Briefly note how ε and α affected learning (e.g., too high ε ⇒ slower convergence; too low α ⇒ sluggish updates).
(These meet the Task 2 deliverables.) 

In [2]:
# task2_ttt_rl.py
from collections import defaultdict
import random

# --- reuse quick helpers (same definitions as in Task 1) ---
WIN = [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]
def legal(b): return [i for i,c in enumerate(b) if c==' ']
def place(b,i,ch): return b[:i]+ch+b[i+1:]
def cur_player(b): return 'X' if b.count('X')==b.count('O') else 'O'
def winner(b):
    for a,b1,c in WIN:
        s = b[a]+b[b1]+b[c]
        if s=='XXX': return 'X'
        if s=='OOO': return 'O'
    return None
def terminal(b): return winner(b) or ' ' not in b
def is_draw(b): return (winner(b) is None) and (' ' not in b)

# --- environment step (X acts; O = random) ---
def step_X_then_randomO(b, a, rng):
    s1 = place(b,a,'X')
    if terminal(s1):
        w=winner(s1); r = 1 if w=='X' else (-1 if w=='O' else 0)
        return s1, r, True
    om = rng.choice(legal(s1))
    s2 = place(s1,om,'O')
    if terminal(s2):
        w=winner(s2); r = 1 if w=='X' else (-1 if w=='O' else 0)
        return s2, r, True
    return s2, 0, False

# --- Q-learning ---
def q_learning(episodes=20000, alpha=0.5, gamma=1.0, eps_start=1.0, eps_end=0.05, seed=0):
    rng = random.Random(seed)
    Q = defaultdict(float)  # key: (state, action_idx)

    def eps_greedy(b, eps):
        acts = legal(b)
        if not acts: return None
        if rng.random() < eps:
            return rng.choice(acts)
        # greedy
        best = max(acts, key=lambda a: Q[(b,a)])
        return best

    def greedy_policy_from_Q():
        pi={}
        seen=set()
        # simple forward gen from empty to collect typical states
        def dfs(b):
            if b in seen or terminal(b): 
                seen.add(b); return
            seen.add(b)
            if cur_player(b)=='X':
                acts = legal(b)
                if acts: pi[b] = max(acts, key=lambda a: Q[(b,a)])
            for a in legal(b):
                dfs(place(b,a,cur_player(b)))
        dfs(' '*9)
        return pi

    stats = []  # [(ep, w,d,l)]
    wins=draws=losses=0
    for ep in range(1, episodes+1):
        b = ' '*9
        eps = eps_end + (eps_start-eps_end)*max(0,(episodes-ep))/episodes
        # single episode
        while not terminal(b):
            if cur_player(b)!='X': break  # safety (shouldn't happen at start)
            a = eps_greedy(b, eps)
            if a is None: break
            b2, r, done = step_X_then_randomO(b, a, rng)
            # next state's max over X actions (if X to move next; else value 0 until X's turn)
            if done:
                target = r
            else:
                if cur_player(b2)!='X':
                    # advance O until X (but our env already moved O once)
                    pass
                # estimate with next available X actions
                acts2 = legal(b2)
                next_q = max((Q[(b2,a2)] for a2 in acts2), default=0.0)
                target = r + gamma*next_q
            Q[(b,a)] += alpha*(target - Q[(b,a)])
            b = b2

        # outcome
        w = winner(b)
        if w=='X': wins+=1
        elif w=='O': losses+=1
        else: draws+=1

        if ep%1000==0:
            stats.append((ep,wins,draws,losses))
            wins=draws=losses=0

    return Q, stats, greedy_policy_from_Q()

# --- evaluation helper ---
def evaluate_policy(pi, games=1000, seed=1):
    rng=random.Random(seed)
    wins=draws=losses=0
    for _ in range(games):
        b=' '*9
        while not terminal(b):
            if cur_player(b)=='X':
                a = pi.get(b, None)
                if a is None:
                    acts = legal(b)
                    if not acts: break
                    a = rng.choice(acts)
                b = place(b,a,'X')
            else:
                b = place(b, rng.choice(legal(b)), 'O')
        w=winner(b)
        if w=='X': wins+=1
        elif w=='O': losses+=1
        else: draws+=1
    return wins, draws, losses

if __name__=='__main__':
    # Train RL
    Q, blocks, pi_rl = q_learning()
    print("RL performance by 1k-episode blocks: ep, wins, draws, losses")
    for row in blocks: print(row)

    # (Optional) Quick comparison vs DP if you computed pi_dp in Task 1 and import it.
    # from task1_ttt_dp import value_iteration
    # _, pi_dp = value_iteration()
    # print("Eval RL:", evaluate_policy(pi_rl))
    # print("Eval DP:", evaluate_policy(pi_dp))


RL performance by 1k-episode blocks: ep, wins, draws, losses
(1000, 575, 132, 293)
(2000, 593, 135, 272)
(3000, 600, 141, 259)
(4000, 655, 103, 242)
(5000, 622, 130, 248)
(6000, 653, 121, 226)
(7000, 719, 89, 192)
(8000, 691, 103, 206)
(9000, 738, 92, 170)
(10000, 764, 93, 143)
(11000, 780, 73, 147)
(12000, 793, 73, 134)
(13000, 846, 64, 90)
(14000, 856, 49, 95)
(15000, 875, 56, 69)
(16000, 886, 45, 69)
(17000, 927, 23, 50)
(18000, 956, 12, 32)
(19000, 944, 20, 36)
(20000, 962, 21, 17)


**Hyper-parameters**. I used ε-greedy with ε decaying 1.0→0.05 over 20k episodes, α=0.5, γ=1. A high initial ε improved exploration (discovering center/corner openings) but lowered early win rate; as ε dropped below ~0.2 the policy stabilized and wins increased. With α=0.5, Q values updated quickly without noticeable oscillations; smaller α (e.g., 0.1) learned more slowly, while larger α (>0.7) oscillated more. Using γ=1 worked well for this finite episodic game. Overall, ε-decay + moderate α produced the best learning curve.